In [1]:
import findspark
findspark.init()
findspark.find()

'C:\\work\\spark-2.4.8-bin-hadoop2.7'

In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("AvgFriendsByAge").getOrCreate()

In [3]:
csv=spark.read.csv("fakefriends.csv", inferSchema=True)

In [4]:
csv.show(3)

+---+--------+---+---+
|_c0|     _c1|_c2|_c3|
+---+--------+---+---+
|  0|    Will| 33|385|
|  1|Jean-Luc| 26|  2|
|  2|    Hugh| 55|221|
+---+--------+---+---+
only showing top 3 rows



In [11]:
data=csv.select(['_c2','_c3'])
data.show(3)

+---+---+
|_c2|_c3|
+---+---+
| 33|385|
| 26|  2|
| 55|221|
+---+---+
only showing top 3 rows



## Type-1

In [12]:
from pyspark.sql.functions import lit
sel = data.withColumn("count", lit(1))
sel.show(5)

+---+---+-----+
|_c2|_c3|count|
+---+---+-----+
| 33|385|    1|
| 26|  2|    1|
| 55|221|    1|
| 40|465|    1|
| 68| 21|    1|
+---+---+-----+
only showing top 5 rows



In [13]:
sel.printSchema()

root
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- count: integer (nullable = false)



In [14]:
sel=sel.dropna()
sel.show(3)

+---+---+-----+
|_c2|_c3|count|
+---+---+-----+
| 33|385|    1|
| 26|  2|    1|
| 55|221|    1|
+---+---+-----+
only showing top 3 rows



In [15]:
sel=sel.groupBy("_c2").sum("_c3", "count")
sel.show(5)

+---+--------+----------+
|_c2|sum(_c3)|sum(count)|
+---+--------+----------+
| 31|    2138|         8|
| 65|    1491|         5|
| 53|    1560|         7|
| 34|    1473|         6|
| 28|    2091|        10|
+---+--------+----------+
only showing top 5 rows



In [19]:
sel.select('_c2', (sel["sum(_c3)"]/sel["sum(count)"]).alias('avg')).orderBy("_c2").show(5)

+---+------------------+
|_c2|               avg|
+---+------------------+
| 18|           343.375|
| 19|213.27272727272728|
| 20|             165.0|
| 21|           350.875|
| 22|206.42857142857142|
+---+------------------+
only showing top 5 rows



## Type-2

In [18]:
data.groupBy("_c2").avg("_c3").sort("_c2").show(5)

+---+------------------+
|_c2|          avg(_c3)|
+---+------------------+
| 18|           343.375|
| 19|213.27272727272728|
| 20|             165.0|
| 21|           350.875|
| 22|206.42857142857142|
+---+------------------+
only showing top 5 rows



## Type-3

In [21]:
from pyspark.sql import functions as func

In [24]:
data.groupBy("_c2").agg(func.round(func.avg("_c3"),5)).sort("_c2").show(5)

+---+------------------+
|_c2|round(avg(_c3), 5)|
+---+------------------+
| 18|           343.375|
| 19|         213.27273|
| 20|             165.0|
| 21|           350.875|
| 22|         206.42857|
+---+------------------+
only showing top 5 rows



In [28]:
data.groupBy("_c2").agg(func.round(func.avg("_c3"),2).alias("friends_avg")).sort("_c2").show(5)

+---+-----------+
|_c2|friends_avg|
+---+-----------+
| 18|     343.38|
| 19|     213.27|
| 20|      165.0|
| 21|     350.88|
| 22|     206.43|
+---+-----------+
only showing top 5 rows

